In [1]:
import numpy as np
from IPython.core.debugger import set_trace
from scipy.io import loadmat
import pickle
import os
import sys
import nltk
from scipy import stats
from sklearn.model_selection import KFold
from scipy.stats import spearmanr, pearsonr
import spacy
import matplotlib.pyplot as plt
from tqdm import tqdm
from spacy import displacy
from collections import defaultdict
import torch
from transformers import *
from sklearn.decomposition import PCA

np.random.seed(97)
torch.manual_seed(97)

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [2]:
hp_text = open("chapter9.txt","r").readlines()
hp_text = [i.strip() for i in hp_text]

sent_text = open("sentences.txt", "r").readlines()
all_sentences = [""]
for i in range(len(sent_text)):
    sent_text[i] = sent_text[i].strip()
    if len(sent_text[i]) == 0 or sent_text[i] == "+":
        all_sentences.append("")
    elif sent_text[i]!="+":
        all_sentences[-1] += sent_text[i] + " "

all_sentences_clean = [i.strip() for i in all_sentences if len(i.strip()) > 0]
all_sentences = all_sentences_clean

In [3]:
all_contexts = []
max_len = 0

for sent in all_sentences:
    words = sent.strip().split(" ")
    max_len = max(max_len, len(words))
    for i in range(len(words)):
        if (len(all_contexts) != 0  and hp_text[len(all_contexts)] == "+"):
            all_contexts.append("")
        if (len(all_contexts) != 0  and hp_text[len(all_contexts)] == "+"):
            all_contexts.append("")
        all_contexts.append(" ".join(words[:i+1]))
            
all_contexts.append([[""]])

In [4]:
print(len(all_contexts))

5176


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertModel.from_pretrained('bert-large-cased', output_hidden_states=True)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [6]:
all_embeddings = []
selected_layer = 11

for i in range(len(all_contexts)):
    print(hp_text[i])
    print(all_contexts[i])
    if hp_text[i] == "+":
        all_embeddings.append(np.zeros(1024))
    else:
        input_ids = torch.tensor([tokenizer.encode(all_contexts[i], add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        all_outputs = model(input_ids)
        all_hidden_states = all_outputs[2]
        selected_hidden_states = all_hidden_states[selected_layer]
        selected_hidden_states = selected_hidden_states.reshape((selected_hidden_states.shape[1], selected_hidden_states.shape[2]))
        mean_hidden_states = selected_hidden_states.mean(axis = 0).detach().numpy()
        all_embeddings.append(mean_hidden_states)

Harry
Harry
had
Harry had
never
Harry had never
believed
Harry had never believed
he
Harry had never believed he
would
Harry had never believed he would
meet
Harry had never believed he would meet
a
Harry had never believed he would meet a
boy
Harry had never believed he would meet a boy
he
Harry had never believed he would meet a boy he
hated
Harry had never believed he would meet a boy he hated
more
Harry had never believed he would meet a boy he hated more
than
Harry had never believed he would meet a boy he hated more than
Dudley,
Harry had never believed he would meet a boy he hated more than Dudley,
but
Harry had never believed he would meet a boy he hated more than Dudley, but
that
Harry had never believed he would meet a boy he hated more than Dudley, but that
was
Harry had never believed he would meet a boy he hated more than Dudley, but that was
before
Harry had never believed he would meet a boy he hated more than Dudley, but that was before
he
Harry had never believed he wo

+

Malfoy
Malfoy
certainly
Malfoy certainly
did
Malfoy certainly did
talk
Malfoy certainly did talk
about
Malfoy certainly did talk about
flying
Malfoy certainly did talk about flying
a
Malfoy certainly did talk about flying a
lot.
Malfoy certainly did talk about flying a lot.
He
He
complained
He complained
loudly
He complained loudly
about
He complained loudly about
first
He complained loudly about first
years
He complained loudly about first years
never
He complained loudly about first years never
getting
He complained loudly about first years never getting
on
He complained loudly about first years never getting on
the
He complained loudly about first years never getting on the
House
He complained loudly about first years never getting on the House
Quidditch
He complained loudly about first years never getting on the House Quidditch
teams
He complained loudly about first years never getting on the House Quidditch teams
and
He complained loudly about first years never getting on the H

where
Ron couldn't see what was exciting about a game with only one ball where
no
Ron couldn't see what was exciting about a game with only one ball where no
one
Ron couldn't see what was exciting about a game with only one ball where no one
was
Ron couldn't see what was exciting about a game with only one ball where no one was
allowed
Ron couldn't see what was exciting about a game with only one ball where no one was allowed
to
Ron couldn't see what was exciting about a game with only one ball where no one was allowed to
fly.
Ron couldn't see what was exciting about a game with only one ball where no one was allowed to fly.
Harry
Harry
had
Harry had
caught
Harry had caught
Ron
Harry had caught Ron
prodding
Harry had caught Ron prodding
Dean's
Harry had caught Ron prodding Dean's
poster
Harry had caught Ron prodding Dean's poster
of
Harry had caught Ron prodding Dean's poster of
West
Harry had caught Ron prodding Dean's poster of West
Ham
Harry had caught Ron prodding Dean's poster of 

KeyboardInterrupt: 

In [ ]:
np.array(all_embeddings).shape

In [8]:
all_embeddings = np.load(os.path.join(save_dir,"incremental_bert_embeddings_layer12.npy"))
pca = PCA(n_components=15)
np.random.seed(97)

reduced = pca.fit_transform(np.array(all_embeddings))
np.save(os.path.join(save_dir,"incremental_bert_embeddings_layer12_PCA_dims_15"), reduced)

In [9]:
reduced.shape

(5176, 15)